Scenario:
    1. Temp View and table creation from dataset
    2.

In [3]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [4]:
if __name__ == "__main__" :
    spark=(SparkSession
           .builder
           .appName("Prg6_DF_temp_views_tables")
           .getOrCreate()
          )

23/05/07 19:31:11 WARN Utils: Your hostname, hps17 resolves to a loopback address: 127.0.1.1; using 192.168.236.132 instead (on interface ens33)
23/05/07 19:31:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/07 19:31:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
parquet_dataset="/home/hari/Downloads/LearningSparkV2-master/parquet_files/userdata1.parquet"
df=spark.read.format("parquet").load(parquet_dataset)
df.show(2)
df.printSchema()

+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
|  registration_dttm| id|first_name|last_name|           email|gender|    ip_address|              cc|  country|birthdate|   salary|           title|comments|
+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
|2016-02-03 13:25:29|  1|    Amanda|   Jordan|ajordan0@com.com|Female|   1.197.201.2|6759521864920116|Indonesia| 3/8/1971| 49756.53|Internal Auditor|   1E+02|
|2016-02-03 22:34:03|  2|    Albert|  Freeman| afreeman1@is.gd|  Male|218.111.175.34|                |   Canada|1/16/1968|150280.17|   Accountant IV|        |
+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
only showing top 2 rows

root
 |-- registratio

In [6]:
#Managed temp view creation

df.createOrReplaceTempView("v_customer_details")

In [7]:
#Querying temporary view

spark.sql("select count(*) from v_customer_details").show()

df_cnty_gen_grd=spark.sql("select country,gender,count(*) from v_customer_details group by country,gender order by country")


+--------+
|count(1)|
+--------+
|    1000|
+--------+



In [10]:
df_cnty_gen_grd.show(10)

df_cnty_gen_grd.printSchema()


+-------------+------+--------+
|      country|gender|count(1)|
+-------------+------+--------+
|     "Bonaire|  Male|       1|
|  Afghanistan|  Male|       1|
|  Afghanistan|Female|       1|
|Aland Islands|  Male|       1|
|      Albania|Female|       3|
|       Angola|Female|       1|
|    Argentina|Female|       9|
|    Argentina|  Male|       4|
|      Armenia|Female|       2|
|      Armenia|  Male|       2|
+-------------+------+--------+
only showing top 10 rows

root
 |-- country: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- count(1): long (nullable = false)



In [15]:
df_cust_dt=(df.withColumn("age",F.round((F.datediff(F.current_date(),F.to_date(F.col("birthdate"),format="M/d/yyyy")))/365,2))
         .select(F.col("ID").alias("cust_id"),"title","first_name","last_name","salary","gender",F.col("birthdate").alias("dob"),"age")
         .where(F.trim("title")!='')
         .orderBy("ID")
        )
(df_cust_dt.write
           .saveAsTable("customer_table_tmp")
)

#temp table creation from dataframe

spark.sql("select count(*) from customer_table_tmp").show(2)



+--------+
|count(1)|
+--------+
|     803|
+--------+



In [17]:
spark.catalog.listTables()

[Table(name='customer_table_tmp', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='v_customer_details', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [19]:
#Create a new database

spark.sql("create database hps")
spark.sql("use hps")

AnalysisException: Database 'hps' already exists

In [22]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/home/hari/workspace/pyspark_codebase/spark-warehouse'),
 Database(name='hps', description='', locationUri='file:/home/hari/workspace/pyspark_codebase/spark-warehouse/hps.db')]

In [23]:
#Creating Global temp view and table

df_cust_dt.createOrReplaceGlobalTempView("v_cust_dt_global_tmp_view")
spark.catalog.listTables()

[Table(name='v_customer_details', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [24]:
df_cust_dt.createOrReplaceGlobalTempView("v_cust_dt_global_tmp_view1")

In [25]:
spark.catalog.listTables()

[Table(name='v_customer_details', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [26]:
spark.catalog.dropTempView("v_customer_details")

In [27]:
spark.catalog.listTables()

[]